In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings("ignore")

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# prompt: pip install scikit-learn-intelex

!pip install scikit-learn-intelex

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.4/91.4 MB 6.4 MB/s eta 0:00:00


In [3]:
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [4]:
df = pd.read_csv("/content/fertilizer datasheet.csv")

In [5]:
# prompt: print all the fertilizer name in the dataset

print(df['Fertilizer'].unique())

['Urea Fertilizer' '28-28 Fertilizer' '17-17-17 Fertilizer'
 '14-35-14 Fertilizer' 'DAP (Diammonium phosphate) Fertilizer'
 '20-20 Fertilizer' '10-26-26 Fertilizer'
 'DAP(Diammonium phosphate) Fertilizer']


In [6]:
df.describe()

,Temparature,Humidity,Moisture,Nitrogen,Potassium,Phosphorous
count,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000
mean,30.894737,60.105263,46.921053,18.894737,4.000000,16.684211
std,3.458664,5.822038,8.520739,11.535811,5.997792,12.497640
min,25.000000,50.000000,25.000000,5.000000,0.000000,0.000000
25%,28.000000,54.000000,42.000000,10.000000,0.000000,9.000000
50%,30.000000,60.000000,48.000000,12.500000,0.000000,16.500000
75%,34.000000,65.000000,53.000000,24.000000,8.000000,28.000000
max,38.000000,72.000000,60.000000,41.000000,19.000000,41.000000


In [7]:
df['Soil_Type'].unique()

array(['Sandy', 'Clayey', 'Loamy', 'Black'], dtype=object)

In [8]:
y = df['Fertilizer'].copy()
X = df.drop('Fertilizer', axis=1).copy()

In [9]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [3,4])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [10]:
X[0]

array([ 0.,  0.,  0.,  1.,  1.,  0.,  0.,  0., 26., 52., 38., 37.,  0.,
        0.])

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=42)

In [12]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [13]:
X_train[0]

array([-0.37387825, -0.58461282,  1.05830052, -0.42163702, -0.42163702,
       -0.58461282,  1.33771211, -0.55555556, -0.31168974, -0.06730769,
        1.31819066, -0.81919915,  0.4363513 ,  1.45823705])

In [14]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators= 100, criterion = 'gini' , random_state= 42)
classifier.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [15]:
y_pred = classifier.predict(X_test)

In [16]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[16  0  0  0  0  0  0]
 [ 0  2  0  0  0  0  0]
 [ 0  0  4  0  0  0  0]
 [ 0  0  0  8  0  0  0]
 [ 0  0  0  0  2  0  0]
 [ 0  0  0  0  0  5  0]
 [ 0  0  0  0  0  0  9]]


1.0

In [17]:
classifier.score(X_test, y_test)

1.0

In [18]:
#encoding the labels for categorical variables
from sklearn.preprocessing import LabelEncoder


In [19]:
#encoding Soil Type variable
encode_soil = LabelEncoder()
df['Soil_Type'] = encode_soil.fit_transform(df['Soil_Type'])

#creating the DataFrame
Soil_Type = pd.DataFrame(zip(encode_soil.classes_,encode_soil.transform(encode_soil.classes_)),columns=['Original','Encoded'])
Soil_Type = Soil_Type.set_index('Original')
Soil_Type

,Encoded
Original,
Black,0
Clayey,1
Loamy,2
Sandy,3


In [20]:
encode_crop =  LabelEncoder()
df['Crop_Type'] = encode_crop.fit_transform(df['Crop_Type'])

#creating the DataFrame
Crop_Type = pd.DataFrame(zip(encode_crop.classes_,encode_crop.transform(encode_crop.classes_)),columns=['Original','Encoded'])
Crop_Type = Crop_Type.set_index('Original')
Crop_Type

,Encoded
Original,
Maize,0
Paddy,1
Sugarcane,2
Wheat,3


In [21]:
encode_ferti = LabelEncoder()
df['Fertilizer'] = encode_ferti.fit_transform(df['Fertilizer'])

#creating the DataFrame
Fertilizer = pd.DataFrame(zip(encode_ferti.classes_,encode_ferti.transform(encode_ferti.classes_)),columns=['Original','Encoded'])
Fertilizer = Fertilizer.set_index('Original')
Fertilizer

,Encoded
Original,
10-26-26 Fertilizer,0
14-35-14 Fertilizer,1
17-17-17 Fertilizer,2
20-20 Fertilizer,3
28-28 Fertilizer,4
DAP (Diammonium phosphate) Fertilizer,5
DAP(Diammonium phosphate) Fertilizer,6
Urea Fertilizer,7


In [22]:
#splitting the data into train and test
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df.drop('Fertilizer',axis=1),df['Fertilizer'],test_size=0.2,random_state=1)
print('Shape of Splitting :')
print('x_train = {}, y_train = {}, x_test = {}, y_test = {}'.format(x_train.shape,y_train.shape,x_test.shape,y_test.shape))

Shape of Splitting :
x_train = (121, 8), y_train = (121,), x_test = (31, 8), y_test = (31,)


In [23]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 121 entries, 19 to 37
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   Temparature  121 non-null    int64
 1   Humidity     121 non-null    int64
 2   Moisture     121 non-null    int64
 3   Soil_Type    121 non-null    int64
 4   Crop_Type    121 non-null    int64
 5   Nitrogen     121 non-null    int64
 6   Potassium    121 non-null    int64
 7   Phosphorous  121 non-null    int64
dtypes: int64(8)
memory usage: 8.0 KB


In [24]:
rand = RandomForestClassifier(random_state = 42)
rand.fit(x_train,y_train)

RandomForestClassifier(random_state=42)

In [25]:
pred_rand = rand.predict(x_test)

In [26]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

params = {
    'n_estimators':[300,400,500],
    'max_depth':[5,10,15],
    'min_samples_split':[2,5,8]
}
grid_rand = GridSearchCV(rand,params,cv=3,verbose=3,n_jobs=-1)

grid_rand.fit(x_train,y_train)

pred_rand = grid_rand.predict(x_test)

print(classification_report(y_test,pred_rand))

print('Best score : ',grid_rand.best_score_)
print('Best params : ',grid_rand.best_params_)

Fitting 3 folds for each of 27 candidates, totalling 81 fits
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         4
           2       1.00      1.00      1.00         3
           3       1.00      1.00      1.00        12
           4       1.00      1.00      1.00         3
           5       1.00      1.00      1.00         1
           6       1.00      1.00      1.00         1
           7       1.00      1.00      1.00         5

    accuracy                           1.00        31
   macro avg       1.00      1.00      1.00        31
weighted avg       1.00      1.00      1.00        31

Best score :  0.9833333333333334
Best params :  {'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 300}


In [27]:
y_train[2]

4

In [28]:
#pickling the file
import pickle
pickle_out = open('classifier.pkl','wb')
pickle.dump(grid_rand,pickle_out)
pickle_out.close()

In [29]:
df.head()

,Temparature,Humidity,Moisture,Soil_Type,Crop_Type,Nitrogen,Potassium,Phosphorous,Fertilizer
0,26,52,38,3,0,37,0,0,7
1,26,52,44,3,0,23,0,20,4
2,30,60,47,3,0,22,0,21,4
3,31,62,48,3,0,14,15,12,2
4,33,64,51,3,0,5,9,29,1


In [30]:
model = pickle.load(open('classifier.pkl','rb'))
ans = model.predict([[34,65,62	,0,	1,	7,	0,	30]])
if ans[0] == 0:
    print("10-26-26 Fertilizer")
elif ans[0] ==1:
    print("14-35-14 Fertilizer")
elif ans[0] == 2:
    print("17-17-17 Fertilizer")
elif ans[0] == 3:
    print("20-20 Fertilizer")
elif ans[0] == 4:
    print("28-28 Fertilizer")
elif ans[0] == 5:
    print("DAP (Diammonium phosphate) Fertilizer")
else:
    print("Invalid inputs")

20-20 Fertilizer
